In [1]:
# Importamos las librerías que necesitamos

# Librerías para extracción de datos y scraping web
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup  # Para analizar y extraer datos de HTML y XML
import requests                # Para realizar solicitudes HTTP y obtener datos de páginas web
import tqdm                    # Para mostrar barras de progreso en procesos largos (útil en bucles de scraping)

# Librerías para el tratamiento y manipulación de datos
# -----------------------------------------------------------------------
import pandas as pd             # Para manipular estructuras de datos como DataFrames
import numpy as np              # Para cálculos numéricos y manejo de arrays
import datetime                 # Para manipulación de fechas y tiempos

# Manejo del tiempo y generación de pausas
# -----------------------------------------------------------------------
from time import sleep          # Pausar la ejecución del código durante un tiempo definido
import random                   # Generar valores aleatorios, útil para espaciar solicitudes web y evitar bloqueos

# Manejo de variables de entorno y archivos dotenv (para tokens y claves)
# -----------------------------------------------------------------------
import os                       # Para interactuar con el sistema de archivos y gestionar variables de entorno
import dotenv                   # Para cargar variables de entorno desde un archivo .env
dotenv.load_dotenv(dotenv_path="../src/.env")  # Cargar las variables de entorno desde un archivo .env

# Importación de módulos desde rutas recursivas
# -----------------------------------------------------------------------
import sys
sys.path.append("../")  # Añadir rutas externas para poder importar módulos desde carpetas superiores
from src.SupportVuelos import get_airport, aiports_to_df, get_flights  # Funciones personalizadas para obtener datos de vuelos

# Para guardar listas en formato JSON cuando hay errores o resultados parciales
# -----------------------------------------------------------------------
import json                     # Para manipular y guardar datos en formato JSON


In [2]:
aeropuertos = ["madrid","niza","oslo"]
resultados = [get_airport(i) for i in aeropuertos]

In [3]:
resultados

[['MAD', '95565077', 'AIRPORT', 'Madrid-Barajas'],
 ['NCE', '95673860', 'AIRPORT', 'Niza'],
 ['OSL', '128667763', 'AIRPORT', 'Oslo-Gardermoen']]

In [4]:
df_resultados = aiports_to_df(resultados)
df_resultados

,skyId,entityId,flightPlaceType,localizedName
0,MAD,95565077,AIRPORT,Madrid-Barajas
1,NCE,95673860,AIRPORT,Niza
2,OSL,128667763,AIRPORT,Oslo-Gardermoen


In [5]:
df_resultados.to_csv("../datos/aeropuertos.csv")

In [2]:
df_resultados = pd.read_csv("../datos/aeropuertos.csv",index_col="Unnamed: 0")

In [3]:
df_resultados

,skyId,entityId,flightPlaceType,localizedName
0,MAD,95565077,AIRPORT,Madrid-Barajas
1,NCE,95673860,AIRPORT,Niza
2,OSL,128667763,AIRPORT,Oslo-Gardermoen


In [4]:
df_flights = get_flights(df_resultados)

Buscando Primer Destino


100%|██████████| 20/20 [02:35<00:00,  7.79s/it]


Buscando Segundo Destino


100%|██████████| 20/20 [02:27<00:00,  7.36s/it]


In [15]:
df_flights.sample(5)

,clase,precio (€),origen_ida,destino_ida,duracion_ida (min),fecha_salida_ida,fecha_llegada_ida,aerolínea_ida,origen_vuelta,destino_vuelta,duracion_vuelta (min),fecha_salida_vuelta,fecha_llegada_vuelta,aerolínea_vuelta
185,business,3693,Madrid,Oslo,315,2024-10-31 17:35:00,2024-10-31 22:50:00,Brussels Airlines,Oslo,Madrid,360,2024-11-03 09:35:00,2024-11-03 15:35:00,Lufthansa
231,economy,682,Madrid,Oslo,350,2024-11-14 12:05:00,2024-11-14 17:55:00,Lufthansa,Oslo,Madrid,360,2024-11-17 09:35:00,2024-11-17 15:35:00,Lufthansa
286,business,1844,Madrid,Oslo,500,2024-11-21 06:15:00,2024-11-21 14:35:00,Lufthansa,Oslo,Madrid,455,2024-11-24 16:20:00,2024-11-24 23:55:00,Lufthansa
160,economy,1333,Madrid,Oslo,225,2024-10-31 18:10:00,2024-10-31 21:55:00,Norwegian,Oslo,Madrid,430,2024-11-03 06:30:00,2024-11-03 13:40:00,Norwegian
145,premium_economy,7058,Madrid,Oslo,380,2024-10-24 10:15:00,2024-10-24 16:35:00,Finnair,Oslo,Madrid,405,2024-10-27 13:05:00,2024-10-27 19:50:00,Scandinavian Airlines


In [17]:
df_flights.to_csv("../datos/vuelos_unidos.csv")

In [20]:
df = pd.read_csv("../datos/vuelos_unidos.csv",index_col="Unnamed: 0")

In [24]:
# Filtrar los vuelos de Madrid a Nice y de Madrid a Oslo
df_niza = df[(df["origen_ida"] == "Madrid") & (df["destino_ida"] == "Nice")]
df_oslo = df[(df["origen_ida"] == "Madrid") & (df["destino_ida"] == "Oslo")]


In [29]:
df_niza.sample(5)

,clase,precio (€),origen_ida,destino_ida,duracion_ida (min),fecha_salida_ida,fecha_llegada_ida,aerolínea_ida,origen_vuelta,destino_vuelta,duracion_vuelta (min),fecha_salida_vuelta,fecha_llegada_vuelta,aerolínea_vuelta
9,economy,2357,Madrid,Nice,115,2024-10-24 17:20:00,2024-10-24 19:15:00,Iberia,Nice,Madrid,120,2024-10-27 12:35:00,2024-10-27 14:35:00,Iberia
64,business,2101,Madrid,Nice,115,2024-11-07 22:00:00,2024-11-07 23:55:00,Iberia,Nice,Madrid,120,2024-11-10 19:00:00,2024-11-10 21:00:00,Iberia
46,business,4493,Madrid,Nice,115,2024-10-31 10:00:00,2024-10-31 11:55:00,Iberia,Nice,Madrid,305,2024-11-03 15:10:00,2024-11-03 20:15:00,Tunisair
72,business,1822,Madrid,Nice,335,2024-11-07 06:25:00,2024-11-07 12:00:00,Brussels Airlines,Nice,Madrid,380,2024-11-10 13:35:00,2024-11-10 19:55:00,Brussels Airlines
119,business,2113,Madrid,Nice,115,2024-11-21 16:20:00,2024-11-21 18:15:00,Iberia,Nice,Madrid,120,2024-11-24 18:50:00,2024-11-24 20:50:00,Iberia


In [33]:
df_oslo.reset_index(inplace=True)
df_oslo.sample(5)

,index,clase,precio (€),origen_ida,destino_ida,duracion_ida (min),fecha_salida_ida,fecha_llegada_ida,aerolínea_ida,origen_vuelta,destino_vuelta,duracion_vuelta (min),fecha_salida_vuelta,fecha_llegada_vuelta,aerolínea_vuelta
142,267,economy,954,Madrid,Oslo,225,2024-11-21 12:20:00,2024-11-21 16:05:00,Norwegian,Oslo,Madrid,350,2024-11-24 11:50:00,2024-11-24 17:40:00,Lufthansa
62,187,business,4506,Madrid,Oslo,455,2024-10-31 15:05:00,2024-10-31 22:40:00,Iberia,Oslo,Madrid,230,2024-11-03 15:00:00,2024-11-03 18:50:00,Iberia
130,255,business,2202,Madrid,Oslo,315,2024-11-14 17:35:00,2024-11-14 22:50:00,Brussels Airlines,Oslo,Madrid,380,2024-11-17 13:45:00,2024-11-17 20:05:00,Lufthansa
9,134,economy,1633,Madrid,Oslo,305,2024-10-24 06:00:00,2024-10-24 11:05:00,KLM,Oslo,Madrid,225,2024-10-27 15:50:00,2024-10-27 19:35:00,Norwegian
49,174,premium_economy,4371,Madrid,Oslo,645,2024-10-31 06:25:00,2024-10-31 17:10:00,Brussels Airlines,Oslo,Madrid,1310,2024-11-03 11:25:00,2024-11-04 09:15:00,Scandinavian Airlines


In [34]:
df_niza.to_csv("../datos/vuelos_niza.csv")
df_oslo.to_csv("../datos/vuelos_oslo.csv")